# **Artificial and Computational Intelligence - Assignment 2 - Group 155**
## **Part A: A Reverse Stones Game with Minimax Algorithm**

### **Objective**
This notebook implements a two-player game where a human plays against a computer opponent. The computer AI is driven by the **Minimax algorithm**, designed to be an unbeatable opponent.

The notebook is structured into three parts:
- **Core Python Logic**: Defines the game's rules and the AI's evaluation strategy.
- **Minimax Algorithm**: Implements the decision-making tree search for the AI.


### static evaluation function
The static evaluation function is designed based on the parity (odd or even) of the total number of stones remaining. While not a perfect strategy, this heuristic provides a simple and effective way for the AI to make logical decisions. The core idea is that controlling the parity can guide the game toward a favorable end-state.

The primary goal of the AI is to force the human player into a situation where they are left with a single stone. By analyzing the parity, the AI can make moves that are more likely to lead to this outcome.

### 2. Scoring Logic
The function uses a two-tiered approach to score a given game state:

Critical End-Game States: The highest and lowest scores are reserved for immediate win/loss conditions, as these are the most important states to recognize.

- return 10: A score of +10 is returned if total_stones is 0. This is a definitive winning state because it means the previous player (the human) was forced to take the last stone.

- return -10: A score of -10 is returned if total_stones is 1. This is a definitive losing state because the current player has no choice but to take the last stone.



In [1]:
import math

def print_piles(piles):
    """Displays the current state of the piles."""
    print("\nCURRENT PILES:")
    for i, count in enumerate(piles):
        print(f"  Pile {i + 1}: {'🔵 ' * count if count > 0 else 'Empty'}")
    print("-" * 20)

def evaluate_state(piles):
    """
    Evaluates the game state based on a heuristic of stone parity (odd/even),
    as suggested by the assignment prompt.
    """
    total_stones = sum(piles)

    # 1. Handle critical end-game states with the highest priority.
    # If only one stone remains, the current player must take it and will lose. This is the worst possible state.
    if total_stones == 1:
        return -10  # Very bad for the current player

    # If zero stones remain, the previous player took the last one. This is a winning state.
    if total_stones == 0:
        return 10   # Very good for the current player

    # 2. Use a heuristic for all other game states.
    # The core idea is that it's advantageous to leave an odd number of stones for the opponent.
    # Therefore, if the current player is facing an even number of stones, they are in a weaker position.
    if total_stones % 2 == 0:
        return -10  # Unfavorable position
    else:
        # If the current player is facing an odd number of stones, they are in a stronger position.
        return 50   # Favorable position

def is_game_over(piles):
    """Checks if the game has ended (all piles are empty)."""
    return all(p == 0 for p in piles)

def get_possible_moves(piles):
    """Generates a list of all valid moves from the current state."""
    moves = []
    for i, count in enumerate(piles):
        if count >= 1:
            moves.append((i, 1))
        if count >= 2:
            moves.append((i, 2))
    return moves

---
### **2. The Minimax Algorithm**
This cell contains the implementation of the **Minimax algorithm**. Minimax is a recursive, decision-making algorithm used in two-player, zero-sum games (like this one).

-   **How it Works:** It simulates every possible move up to a certain `depth`.
    -   The **Maximizer** (our AI) tries to choose the move that leads to the outcome with the highest possible score.
    -   The **Minimizer** (the human player) is assumed to always choose the move that leads to the outcome with the lowest possible score for the AI.
-   **`find_best_move`:** This function acts as the entry point. It iterates through all of the computer's possible first moves and uses the `minimax` function to look ahead and see which of those moves will ultimately lead to the best result, assuming the human plays perfectly against it.

In [2]:
def minimax(piles, depth, is_maximizing_player):
    """
    Minimax algorithm implementation with a fixed depth of 3.
    """
    if is_game_over(piles):
        # In a misère game, if the game is over, the player whose turn it WAS lost.
        # So if it's the maximizer's turn now, the minimizer just moved and lost. Good for maximizer.
        return 100 if is_maximizing_player else -100
    
    if depth == 0:
        return evaluate_state(piles)

    if is_maximizing_player:
        max_eval = -math.inf
        for pile_idx, stones_to_remove in get_possible_moves(piles):
            new_piles = list(piles)
            new_piles[pile_idx] -= stones_to_remove
            evaluation = minimax(new_piles, depth - 1, False)
            max_eval = max(max_eval, evaluation)
        return max_eval
    else:
        min_eval = math.inf
        for pile_idx, stones_to_remove in get_possible_moves(piles):
            new_piles = list(piles)
            new_piles[pile_idx] -= stones_to_remove
            evaluation = minimax(new_piles, depth - 1, True)
            min_eval = min(min_eval, evaluation)
        return min_eval

def find_best_move(piles, depth):
    """
    Determines the computer's best move. As the maximizing player, it chooses
    the move that leads to the state with the highest possible score for itself,
    assuming the opponent will play optimally (and try to minimize the score).
    """
    best_move = None
    best_value = -math.inf # Start at negative infinity because we want to maximize our score.

    for pile_idx, stones_to_remove in get_possible_moves(piles):
        new_piles = list(piles)
        new_piles[pile_idx] -= stones_to_remove
        
        # Look ahead to see what the opponent's best response (lowest score) would be.
        move_value = minimax(new_piles, depth - 1, False) 
        
        # If the outcome of this move is better (higher) for us than any other move seen so far,
        # then this is our new best move.
        if move_value >= best_value:
            best_value = move_value
            best_move = (pile_idx, stones_to_remove)
            
    return best_move

In [3]:
def get_human_move(piles):
    """
    Prompts the human player for their move and performs validation.
    """
    while True:
        try:
            pile_idx = int(input("Select a pile (1, 2, or 3): ")) - 1
            if pile_idx not in [0, 1, 2] or piles[pile_idx] == 0:
                print("Invalid pile selection. Please choose a non-empty pile (1, 2, or 3).")
                continue
            stones = int(input(f"Remove 1 or 2 stones from pile {pile_idx + 1}: "))
            if stones not in [1, 2] or piles[pile_idx] < stones:
                print(f"Invalid number of stones. You can only remove 1 or 2, and the pile only has {piles[pile_idx]}.")
                continue
            return (pile_idx, stones)
        except (ValueError, IndexError):
            print("Invalid input. Please enter valid numbers.")

def game_loop():
    """Main function to run the text-based Reverse Stones Game."""
    piles = [6, 6, 6]
    human_turn = True
    print("\033[1;99m    Welcome to the Reverse Stones Game of Group 155    \033[0m \n")
    print("\033[1;99m Rules:The player who removes the last stone loses.\033[0m")
    while not is_game_over(piles):
        print("-" * 20)
        print_piles(piles)
        if human_turn:
            print("\n \033[1;92mYour turn.\033[0m")
            pile_idx, stones_to_remove = get_human_move(piles)
            piles[pile_idx] -= stones_to_remove
            if is_game_over(piles):
                print("\033[1;92m🏆 Bad Luck...Computer wins!\033[0m \n")
                break
        else:
            print("\nComputer's turn.")
            move = find_best_move(piles, depth=3) # Increased depth for better end-game
            if move:
                pile_idx, stones_to_remove = move
                piles[pile_idx] -= stones_to_remove
                print(f"\033[1;94mComputer removed {stones_to_remove} stone(s) from pile {pile_idx + 1}.\033[0m")
                if is_game_over(piles):
                    print("\nComputer took the last stone!\n")
                    print("\033[1;92m🏆Congratulation!! You wins!\033[0m \n")
                    break
        human_turn = not human_turn
    print("\n \033[1;92m --- Game Over ---\033[0m \n")

# To play the terminal version, run this cell
game_loop()

    Welcome to the Reverse Stones Game of Group 155     

 Rules:The player who removes the last stone loses.
--------------------

CURRENT PILES:
  Pile 1: 🔵 🔵 🔵 🔵 🔵 🔵 
  Pile 2: 🔵 🔵 🔵 🔵 🔵 🔵 
  Pile 3: 🔵 🔵 🔵 🔵 🔵 🔵 
--------------------

 Your turn.


Select a pile (1, 2, or 3):  1
Remove 1 or 2 stones from pile 1:  3


Invalid number of stones. You can only remove 1 or 2, and the pile only has 6.


Select a pile (1, 2, or 3):  1
Remove 1 or 2 stones from pile 1:  2


--------------------

CURRENT PILES:
  Pile 1: 🔵 🔵 🔵 🔵 
  Pile 2: 🔵 🔵 🔵 🔵 🔵 🔵 
  Pile 3: 🔵 🔵 🔵 🔵 🔵 🔵 
--------------------

Computer's turn.
Computer removed 2 stone(s) from pile 3.
--------------------

CURRENT PILES:
  Pile 1: 🔵 🔵 🔵 🔵 
  Pile 2: 🔵 🔵 🔵 🔵 🔵 🔵 
  Pile 3: 🔵 🔵 🔵 🔵 
--------------------

 Your turn.


...

Invalid pile selection. Please choose a non-empty pile (1, 2, or 3).


Select a pile (1, 2, or 3):  1


Invalid pile selection. Please choose a non-empty pile (1, 2, or 3).


Select a pile (1, 2, or 3):  2
Remove 1 or 2 stones from pile 2:  1


--------------------

CURRENT PILES:
  Pile 1: Empty
  Pile 2: 🔵 
  Pile 3: Empty
--------------------

Computer's turn.
Computer removed 1 stone(s) from pile 2.

Computer took the last stone!

🏆Congratulation!! You wins! 


  --- Game Over --- 

